In [4]:
!pip install qiskit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.2 MB/s eta 0:00:00


In [5]:
# -*- coding: utf-8 -*-
%%writefile sdes.py
import itertools

# --- Các bảng hoán vị và S-Box cố định ---
P10 = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
P8 = (6, 3, 7, 4, 8, 5, 10, 9)
P4 = (2, 4, 3, 1)

IP = (2, 6, 3, 1, 4, 8, 5, 7)
IP_inv = (4, 1, 3, 5, 7, 2, 8, 6) # Hoán vị ngược của IP

EP = (4, 1, 2, 3, 2, 3, 4, 1) # Bảng mở rộng/hoán vị E/P

S0 = [[1, 0, 3, 2],
      [3, 2, 1, 0],
      [0, 2, 1, 3],
      [3, 1, 3, 2]]

S1 = [[0, 1, 2, 3],
      [2, 0, 1, 3],
      [3, 0, 1, 0],
      [2, 1, 0, 3]]

# --- Các hàm trợ giúp ---
def permute(k, p):
  """Áp dụng hoán vị p cho chuỗi bit k."""
  return "".join(k[i-1] for i in p)

def left_shift(k, n=1):
  """Dịch trái chuỗi bit k đi n vị trí."""
  L = k[:5]
  R = k[5:]
  L_shifted = L[n:] + L[:n]
  R_shifted = R[n:] + R[:n]
  return L_shifted + R_shifted

def xor(a, b):
  """Thực hiện phép XOR trên hai chuỗi bit."""
  return "".join(str(int(x) ^ int(y)) for x, y in zip(a, b))

def bits_to_int(bits):
    """Chuyển chuỗi bit thành số nguyên."""
    return int(bits, 2)

def int_to_bits(n, length):
    """Chuyển số nguyên thành chuỗi bit có độ dài length."""
    return format(n, f'0{length}b')

# --- Các hàm chính của S-DES ---
def generate_keys(key_10bit):
  """Tạo hai khóa con K1 và K2 từ khóa 10-bit ban đầu."""
  if len(key_10bit) != 10:
    raise ValueError("Khóa phải có độ dài 10 bit")
  # 1. Áp dụng P10
  p10_key = permute(key_10bit, P10)
  # 2. Dịch trái 1 bit (LS-1) cho mỗi nửa
  ls1_key = left_shift(p10_key, 1)
  # 3. Áp dụng P8 để tạo K1
  K1 = permute(ls1_key, P8)
  # 4. Dịch trái 2 bit (LS-2) từ kết quả LS-1
  ls2_key = left_shift(ls1_key, 2)
  # 5. Áp dụng P8 để tạo K2
  K2 = permute(ls2_key, P8)
  return K1, K2

def sbox_lookup(bits, sbox):
  """Tra cứu giá trị trong S-Box."""
  row = bits_to_int(bits[0] + bits[3])
  col = bits_to_int(bits[1] + bits[2])
  val = sbox[row][col]
  return int_to_bits(val, 2) # Trả về chuỗi 2 bit

def function_fk(bits_8, subkey_8):
  """Hàm f_k(L, R, SK) = P4(S0(L_sbox) || S1(R_sbox))
     trong đó L_sbox và R_sbox là kết quả XOR của E/P(R) với SK."""
  # Tách lấy nửa phải (4 bit)
  R = bits_8[4:]
  # 1. Mở rộng/hoán vị E/P
  ep_result = permute(R, EP)
  # 2. XOR với khóa con
  xor_result = xor(ep_result, subkey_8)
  # 3. Chia thành 2 nửa cho S-Box
  left_sbox_input = xor_result[:4]
  right_sbox_input = xor_result[4:]
  # 4. Tra cứu S-Box
  s0_output = sbox_lookup(left_sbox_input, S0)
  s1_output = sbox_lookup(right_sbox_input, S1)
  # 5. Kết hợp kết quả S-Box
  sbox_combined = s0_output + s1_output
  # 6. Áp dụng hoán vị P4
  p4_result = permute(sbox_combined, P4)
  return p4_result

def encrypt(plaintext_8bit, key_10bit):
  """Mã hóa plaintext 8-bit bằng S-DES."""
  if len(plaintext_8bit) != 8:
      raise ValueError("Plaintext phải có độ dài 8 bit")

  K1, K2 = generate_keys(key_10bit)

  # 1. Hoán vị ban đầu (IP)
  ip_result = permute(plaintext_8bit, IP)
  L0, R0 = ip_result[:4], ip_result[4:]

  # --- Vòng 1 ---
  # 2. Áp dụng hàm f_k với K1
  fk_result1 = function_fk(ip_result, K1)
  # 3. XOR kết quả f_k với nửa trái (L0)
  L1 = xor(L0, fk_result1)
  R1 = R0 # Nửa phải giữ nguyên

  # --- Switch (SW) ---
  # 4. Đảo nửa trái và nửa phải
  L2, R2 = R1, L1 # Lưu ý: bây giờ là R1, L1

  # --- Vòng 2 ---
  # 5. Áp dụng hàm f_k với K2
  #    Đầu vào cho fk bây giờ là (L2 || R2) = (R1 || L1)
  fk_input2 = L2 + R2
  fk_result2 = function_fk(fk_input2, K2)
  # 6. XOR kết quả f_k với nửa trái hiện tại (L2 = R1)
  L3 = xor(L2, fk_result2)
  R3 = R2 # Nửa phải giữ nguyên (R3 = L1)

  # --- Kết hợp và Hoán vị cuối cùng ---
  # 7. Kết hợp L3 và R3
  combined_final = L3 + R3
  # 8. Áp dụng hoán vị ngược IP^-1
  ciphertext_8bit = permute(combined_final, IP_inv)

  return ciphertext_8bit

# --- Ví dụ sử dụng ---
plaintext = "10101010"
key = "1010000010"

try:
    k1, k2 = generate_keys(key)
    print(f"Khóa ban đầu: {key}")
    print(f"Khóa con K1: {k1}")
    print(f"Khóa con K2: {k2}")

    ciphertext = encrypt(plaintext, key)
    print(f"\nPlaintext : {plaintext}")
    print(f"Ciphertext: {ciphertext}")

    # Đối chiếu: Nếu bạn có một cặp plaintext/ciphertext đã biết
    # known_plaintext = "..."
    # known_ciphertext = "..."
    # calculated_ciphertext = encrypt(known_plaintext, key)
    # assert calculated_ciphertext == known_ciphertext
    # print("Đối chiếu thành công!")

except ValueError as e:
    print(f"Lỗi: {e}")

Overwriting sdes.py


In [6]:
%%writefile sdes.py
# -*- coding: utf-8 -*-
# --- Các bảng hoán vị và S-Box cố định ---
P10 = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
P8  = (6, 3, 7, 4, 8, 5, 10, 9)
P4  = (2, 4, 3, 1)

IP     = (2, 6, 3, 1, 4, 8, 5, 7)
IP_INV = (4, 1, 3, 5, 7, 2, 8, 6)

EP = (4, 1, 2, 3, 2, 3, 4, 1)

S0 = [
    [1,0,3,2],
    [3,2,1,0],
    [0,2,1,3],
    [3,1,3,2]
]
S1 = [
    [0,1,2,3],
    [2,0,1,3],
    [3,0,1,0],
    [2,1,0,3]
]

def permute(bits, table):
    return ''.join(bits[i-1] for i in table)

def left_shift(bits, n):
    L, R = bits[:5], bits[5:]
    return L[n:]+L[:n] + R[n:]+R[:n]

def xor(a, b):
    return ''.join(str(int(x)^int(y)) for x,y in zip(a,b))

def bits_to_int(bits):
    return int(bits, 2)

def int_to_bits(val, length):
    return format(val, f'0{length}b')

def generate_keys(key10):
    if len(key10)!=10:
        raise ValueError("Key phải 10 bit")
    p10 = permute(key10, P10)
    ls1 = left_shift(p10, 1)
    K1  = permute(ls1, P8)
    ls2 = left_shift(ls1, 2)
    K2  = permute(ls2, P8)
    return K1, K2

def sbox_lookup(bits, sbox):
    row = bits_to_int(bits[0]+bits[3])
    col = bits_to_int(bits[1]+bits[2])
    return int_to_bits(sbox[row][col], 2)

def function_fk(bits8, subkey):
    L, R = bits8[:4], bits8[4:]
    ep = permute(R, EP)
    x  = xor(ep, subkey)
    l2 = sbox_lookup(x[:4], S0)
    r2 = sbox_lookup(x[4:],   S1)
    return permute(l2+r2, P4)

def encrypt(pt8, key10):
    if len(pt8)!=8:
        raise ValueError("Plaintext phải 8 bit")
    K1, K2 = generate_keys(key10)
    ip      = permute(pt8, IP)
    L, R    = ip[:4], ip[4:]
    # Round 1
    f1 = function_fk(L+R, K1)
    L1 = xor(L, f1)
    # Swap
    L2, R2 = R, L1
    # Round 2
    f2 = function_fk(L2+R2, K2)
    L3 = xor(L2, f2)
    # Final
    return permute(L3+R2, IP_INV)

def decrypt(ct8, key10):
    if len(ct8)!=8:
        raise ValueError("Ciphertext phải 8 bit")
    K1, K2 = generate_keys(key10)
    ip      = permute(ct8, IP)
    L, R    = ip[:4], ip[4:]
    # Round 1 với K2
    f1 = function_fk(L+R, K2)
    L1 = xor(L, f1)
    # Swap
    L2, R2 = R, L1
    # Round 2 với K1
    f2 = function_fk(L2+R2, K1)
    L3 = xor(L2, f2)
    # Final
    return permute(L3+R2, IP_INV)


Overwriting sdes.py


In [7]:
%%bash
cat > test_sdes.py <<'EOF'
import random
import pytest
from sdes import encrypt, decrypt

@pytest.mark.parametrize("_", range(10))
def test_random_vectors(_):
    plaintext = ''.join(random.choice('01') for _ in range(8))
    key = ''.join(random.choice('01') for _ in range(10))
    cipher = encrypt(plaintext, key)
    assert decrypt(cipher, key) == plaintext
EOF


In [8]:
!pytest -q


..........                                                               [100%]
10 passed in 0.07s


In [3]:
%%writefile sdes.py
# -*- coding: utf-8 -*-
# --- Các bảng hoán vị và S-Box cố định ---
P10 = (3, 5, 2, 7, 4, 10, 1, 9, 8, 6)
P8  = (6, 3, 7, 4, 8, 5, 10, 9)
P4  = (2, 4, 3, 1)

IP     = (2, 6, 3, 1, 4, 8, 5, 7)
IP_INV = (4, 1, 3, 5, 7, 2, 8, 6)

EP = (4, 1, 2, 3, 2, 3, 4, 1)

S0 = [
    [1,0,3,2],
    [3,2,1,0],
    [0,2,1,3],
    [3,1,3,2]
]
S1 = [
    [0,1,2,3],
    [2,0,1,3],
    [3,0,1,0],
    [2,1,0,3]
]

# --- Hàm tiện ích ---
def permute(bits, table):
    return ''.join(bits[i-1] for i in table)

def left_shift(bits, n):
    L, R = bits[:5], bits[5:]
    return L[n:]+L[:n] + R[n:]+R[:n]

def xor(a, b):
    return ''.join(str(int(x)^int(y)) for x,y in zip(a,b))

def bits_to_int(bits):
    return int(bits, 2)

def int_to_bits(val, length):
    return format(val, f'0{length}b')

# --- Sinh khóa ---
def generate_keys(key10):
    if len(key10)!=10:
        raise ValueError("Key phải 10 bit")
    p10 = permute(key10, P10)
    ls1 = left_shift(p10, 1)
    K1  = permute(ls1, P8)
    ls2 = left_shift(ls1, 2)
    K2  = permute(ls2, P8)
    return K1, K2

# --- Feistel function ---
def sbox_lookup(bits, sbox):
    row = bits_to_int(bits[0]+bits[3])
    col = bits_to_int(bits[1]+bits[2])
    return int_to_bits(sbox[row][col], 2)

def function_fk(bits8, subkey):
    L, R = bits8[:4], bits8[4:]
    ep = permute(R, EP)
    x  = xor(ep, subkey)
    l2 = sbox_lookup(x[:4], S0)
    r2 = sbox_lookup(x[4:],   S1)
    return permute(l2+r2, P4)

# --- Encrypt ---
def encrypt(pt8, key10):
    if len(pt8)!=8:
        raise ValueError("Plaintext phải 8 bit")
    K1, K2 = generate_keys(key10)
    ip      = permute(pt8, IP)
    L, R    = ip[:4], ip[4:]

    # Round 1
    f1 = function_fk(L+R, K1)
    L1 = xor(L, f1)
    # Swap
    L2, R2 = R, L1
    # Round 2
    f2 = function_fk(L2+R2, K2)
    L3 = xor(L2, f2)
    # Final
    return permute(L3+R2, IP_INV)

# --- Decrypt (K2 trước, K1 sau) ---
def decrypt(ct8, key10):
    if len(ct8)!=8:
        raise ValueError("Ciphertext phải 8 bit")
    K1, K2 = generate_keys(key10)
    ip      = permute(ct8, IP)
    L, R    = ip[:4], ip[4:]

    # Round 1 với K2
    f1 = function_fk(L+R, K2)
    L1 = xor(L, f1)
    # Swap
    L2, R2 = R, L1
    # Round 2 với K1
    f2 = function_fk(L2+R2, K1)
    L3 = xor(L2, f2)
    # Final
    return permute(L3+R2, IP_INV)


Writing sdes.py


In [2]:
%%writefile mock_oracle.py
from qiskit import QuantumCircuit

def get_mock_oracle(n_qubits: int, target_index: int) -> QuantumCircuit:
    """
    Trả về một oracle circuit trên `n_qubits`, thực hiện phase flip:
      |target_index⟩ → -|target_index⟩
    """
    qc = QuantumCircuit(n_qubits)
    # 1. Chuẩn hóa: X lên tất cả qubit có bit = 0 trong target_index
    bstr = format(target_index, f'0{n_qubits}b')
    for i, bit in enumerate(bstr):
        if bit == '0':
            qc.x(i)
    # 2. Multi-controlled Z (phase flip) trên |11…1⟩
    qc.h(n_qubits-1)
    qc.mcx(list(range(n_qubits-1)), n_qubits-1)
    qc.h(n_qubits-1)
    # 3. Khôi phục X
    for i, bit in enumerate(bstr):
        if bit == '0':
            qc.x(i)
    return qc


Writing mock_oracle.py


In [13]:
%%writefile real_oracle.py
from qiskit import QuantumCircuit, QuantumRegister
from sdes import encrypt

def build_sdes_subcircuit(key10: str) -> QuantumCircuit:
    qc = QuantumCircuit(16, name="sdes_enc")
    for i in range(256):
        pt_bits = format(i, '08b')
        ct_bits = encrypt(pt_bits, key10)
        ctrl_bits = [j for j, b in enumerate(pt_bits) if b == '1']
        if ctrl_bits:
            for j, b in enumerate(ct_bits):
                if b == '1':
                    qc.mcx(ctrl_bits, [8 + j])
                    qc.x(8 + j)
                    qc.mcx(ctrl_bits, [8 + j])
                    qc.x(8 + j)
    return qc

def get_sdes_oracle(key10: str) -> QuantumCircuit:
    target_ct = "10100001"

    qr_pt = QuantumRegister(8, "pt")
    qr_ct = QuantumRegister(8, "ct")
    qr_flag = QuantumRegister(1, "flag")
    qc = QuantumCircuit(qr_pt, qr_ct, qr_flag)

    subcirc = build_sdes_subcircuit(key10)
    qc.append(subcirc, qr_pt[:] + qr_ct[:])

    for i, b in enumerate(target_ct):
        if b == '0':
            qc.x(qr_ct[i])

    qc.mcx(qr_ct[:], qr_flag[0])

    for i, b in enumerate(target_ct):
        if b == '0':
            qc.x(qr_ct[i])

    qc.append(subcirc.inverse(), qr_pt[:] + qr_ct[:])

    return qc


Overwriting real_oracle.py


In [14]:
from mock_oracle import get_mock_oracle
qc1 = get_mock_oracle(3, target_index=5)
#qc1.draw('mpl')
print(qc1.draw(output='text'))

from real_oracle import get_sdes_oracle
qc2 = get_sdes_oracle("1010000010")
#qc2.draw("mpl")
print(qc2.draw(output='text'))


                    
q_0: ───────■───────
     ┌───┐  │  ┌───┐
q_1: ┤ X ├──■──┤ X ├
     ├───┤┌─┴─┐├───┤
q_2: ┤ H ├┤ X ├┤ H ├
     └───┘└───┘└───┘
      ┌────────────┐               ┌───────────────┐
pt_0: ┤0           ├───────────────┤0              ├
      │            │               │               │
pt_1: ┤1           ├───────────────┤1              ├
      │            │               │               │
pt_2: ┤2           ├───────────────┤2              ├
      │            │               │               │
pt_3: ┤3           ├───────────────┤3              ├
      │            │               │               │
pt_4: ┤4           ├───────────────┤4              ├
      │            │               │               │
pt_5: ┤5           ├───────────────┤5              ├
      │            │               │               │
pt_6: ┤6           ├───────────────┤6              ├
      │            │               │               │
pt_7: ┤7           ├───────────────┤7              ├
     

In [44]:
import sys
!{sys.executable} -m pip install pylatexenc

In [45]:
pip install pylatexenc
